# Обучение моделей скорера

В этом нотбуке будет произведен сбор обучающей выборки и обучение скорера на основе ranking SVM, CatBoost использованием дополнительных признаков.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import json
import pickle
import re
from copy import copy, deepcopy
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker import *
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)
from src.evaluation.spell_ru_eval import align_sents

from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import catboost
from catboost import CatBoost, Pool, MetricVisualizer

from IPython.display import display
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Нахождение корректных токенов

Начнем с того, что найдем какие токены в каждой позиции правильные. Так как наша модель не умеет никак объединять токены надо будет найти те предложения, где токены исходного предложения не объединяются.

Создадим токенизаторы.

In [4]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x, escape=False)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

Прочитаем все предложения.

In [5]:
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    sentences_corrected = inf.readlines()

Возьмем какое-либо случайное предложение на данный момент.

In [6]:
np.random.seed(42)
idx = np.random.randint(0, len(sentences))

In [7]:
sentence = sentences[idx]
sentence_corrected = sentences_corrected[idx]
print(sentence)
print(sentence_corrected)

Намазывем уже остывший корж " кремом " ( " фромаж блан " или творог, риккота, протертые сквозь мелкое сито, даже густая сметана подойдет ), совсем немного, только, чтобы ягоды потом прилипли.

Намазываем уже остывший корж кремом фромаж блан или творог риккота протертые сквозь мелкое сито даже густая сметана подойдет совсем немного только чтобы ягоды потом прилипли



In [8]:
tokenized_sentence_raw = tokenizer(
    sentence.lower().replace('ё', 'е')
)
tokenized_sentence_corrected = tokenizer(
    sentence_corrected.lower().replace('ё', 'е')
)

Уберем пунктуацию из изначального предложения.

In [9]:
tokenized_sentence = []
indices_mapping = []
for i, token in enumerate(tokenized_sentence_raw):
    if not re.fullmatch(f'[{punctuation}]+', token):
        tokenized_sentence.append(token)
        indices_mapping.append(i)

Пробуем убрать пунктуацию и вывести выравнивание.

In [10]:
alignment = align_sents(tokenized_sentence, tokenized_sentence_corrected)
for pair in alignment:
    left_indices, right_indices = pair
    print(f'{tokenized_sentence[left_indices[0]:left_indices[1]]}\t'
          f'{tokenized_sentence_corrected[right_indices[0]:right_indices[1]]}')

['намазывем']	['намазываем']
['уже']	['уже']
['остывший']	['остывший']
['корж']	['корж']
['кремом']	['кремом']
['фромаж']	['фромаж']
['блан']	['блан']
['или']	['или']
['творог']	['творог']
['риккота']	['риккота']
['протертые']	['протертые']
['сквозь']	['сквозь']
['мелкое']	['мелкое']
['сито']	['сито']
['даже']	['даже']
['густая']	['густая']
['сметана']	['сметана']
['подойдет']	['подойдет']
['совсем']	['совсем']
['немного']	['немного']
['только']	['только']
['чтобы']	['чтобы']
['ягоды']	['ягоды']
['потом']	['потом']
['прилипли']	['прилипли']


Как видим, тут выравнивание оказалось очень простым: 1 к 1. Таким образом, для каждой позиции слева мы нашли корректный токен справа. На пунктуацию можем внимания не обращать, потому что она не может попасть в позиции для исправлений.

Теперь такое надо сделать со всеми предложениями в датасете, но надо учесть, что иногда нам могут встречаться случаи, когда нескольким токенам слева соответствует один токен справа (например, когда в слове случайно вставлен пробел). Мы с таким работать не умеем, а потому будем игнорировать такие предложения.

In [11]:
def find_true_correction(sentence, sentence_corrected):
    """Find correction for sentence."""
    tokenized_sentence_raw = tokenizer(
        sentence.lower().replace('ё', 'е')
    )
    tokenized_sentence_corrected = tokenizer(
        sentence_corrected.lower().replace('ё', 'е')
    )
    # remove punctuation from source sentence and make mapping 
    # to initial indices
    tokenized_sentence = []
    indices_mapping = []
    for i, token in enumerate(tokenized_sentence_raw):
        if not re.fullmatch(f'[{punctuation}]+', token):
            tokenized_sentence.append(token)
            indices_mapping.append(i)
    
    alignment = align_sents(tokenized_sentence, tokenized_sentence_corrected)
    answer = {}
    for i, pair in enumerate(alignment):
        left_indices, right_indices = pair
        if left_indices[1] - left_indices[0] > 1:
            return None
        answer[indices_mapping[left_indices[0]]] = (
            detokenizer(
                 tokenized_sentence_corrected[
                     right_indices[0]:right_indices[1]]
            )
        )
    return answer

Выполняем действие над всеми предложениями в обучающем датасете.

In [12]:
fail_indices = []
succ_indices = []
answers = {}
for i, (sentence, sentence_corrected) in enumerate(
    zip(sentences, sentences_corrected)
):
    answer = find_true_correction(sentence, sentence_corrected)
    if answer is None:
        fail_indices.append(i)
    else:
        succ_indices.append(i)
        answers[i] = answer
        
sentences_to_check = [sentences[idx] for idx in succ_indices]

Посмотрим как много предложений, которые мы не смогли обработать.

In [13]:
num_fails = len(fail_indices)
all_fails = 1727 # знаем исходя из тестировния
print(f'Количество ошибок со слияниями: {num_fails}')
print(f'Доля ошибок со слияниями: {num_fails/all_fails:.3f}')

Количество ошибок со слияниями: 70
Доля ошибок со слияниями: 0.041


Это значение не слишком велико.

## Сбор обучающей выборки

Теперь требуется собрать саму обучающую выборку. 

Пусть на вход подается некоторое предложение для исправления. В процессе работы модели position selector находит позиции для исправления и подает в candidate scorer список кандидатов. Наша задача &mdash; зафиксировать номер выбранной позиции и список пришедших кандидатов вместе с их признаками.

Такое сохранение будет сделано при помощи callback-функции после вызова candidate scorer. Она соберет позиции, кандидатов с признаками, результаты скоринга при помощи BERT и запишет это в файл.

In [ ]:
data = {}

In [ ]:
def create_callback_bert_scorer(num_batch):
    def callback_bert_scorer(
        tokenized_sentences, indices_processing_sentences, 
        candidates, positions, 
        scoring_results, scoring_info
    ):
        for num_sent, candidates_sentence in enumerate(candidates):
            candidates_to_dump = []
            position = positions[num_sent]
            for i, candidate in enumerate(candidates_sentence[position]):
                copy_candidate = copy(candidate)
                copy_candidate.update(scoring_info[num_sent][i])
                candidates_to_dump.append(copy_candidate)

            key = (num_batch, indices_processing_sentences[num_sent])
            if key not in data:
                data[key] = []
            data[key].append({
                'position': position,
                'candidates': candidates_to_dump
            })
    return callback_bert_scorer

Проинициализируем модель.

In [ ]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
handcode_table_path = os.path.join(DATA_PATH, 'processed', 'handcode_table', 
                                   'table.json')
with open(handcode_table_path, 'r') as inf:
    handcode_table = json.load(inf)
candidate_generator = CandidateGenerator(
    words=vocab.keys(), handcode_table=handcode_table, max_distance=1
)

In [ ]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_100.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_100.arpa.binary')
)
margin_border = np.log(2.5)
position_selector = KenlmPositionSelector(
    model_left_right, model_right_left, margin_border=margin_border
)

In [ ]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
bert_scorer_correction = BertScorerCorrection(model, bert_tokenizer)
agg_subtoken_func = 'mean'
bert_scorer = BertScorer(
    bert_scorer_correction, agg_subtoken_func
)
candidate_scorer = CandidateScorer(bert_scorer)

In [ ]:
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    tokenizer,
    detokenizer,
    max_it=max_it
)

Запустим сбор обучающей выборки.

In [ ]:
batch_size = 5
num_batches = int(np.ceil(len(sentences_to_check) // batch_size))

for i in tqdm(range(num_batches)):
    cur_sentences = sentences_to_check[i*batch_size:(i+1)*batch_size]
    spellchecker(
        cur_sentences,
        callback_candidate_scorer=create_callback_bert_scorer(i)
    )

Переведем текущие индексы предложений в исходные и сохраним данные на диск.

In [ ]:
data_adjusted = {}
for key, value in data.items():
    key_adjusted = succ_indices[key[0]*batch_size + key[1]]
    data_adjusted[key_adjusted] = value
    
data = data_adjusted

In [ ]:
data_with_answers = {}
for key, value in data.items():
    new_value = []
    for item in value:
        new_item = copy(item)
        new_item['answer'] = answers[key][item['position']]
        new_value.append(new_item)
    data_with_answers[key] = new_value
    
data = data_with_answers

In [ ]:
!mkdir -p ../data/processed/candidate_scorer

In [ ]:
with open(os.path.join(DATA_PATH, 'processed', 'candidate_scorer', 'data.bin'), 'wb') as ouf:
    pickle.dump(data, ouf)

### Аналитика

Проведем небольшую аналитку по данным. Найдем:
1. Количество предложений без исправлений.
2. Среднее количество исправлений.
3. Доля случаев, когда использовано максимальное число итераций.
4. Доля случаев, когда нет корректного токена в списке кандидатов
5. Среднее количество кандидатов.
6. Доля случаев, когда надо оставить изначальный токен при изменении.

In [14]:
with open(os.path.join(DATA_PATH, 'processed', 'candidate_scorer', 'data.bin'), 'rb') as inf:
    data = pickle.load(inf)

1). Количество предложений без исправлений.

In [15]:
len(sentences_to_check) - len(data.keys())

582

Как видим, почти четверть предложений остались без исправлений. 

Исходя из статья, посвященной соревнованию, данные которого мы используем, на валидацию и тест было отведено в сумме 1600 корректных предложений. Они разделялись поровно и случайно, поэтому нет гарантий, что удастся получить ровно 800. Следует так же учесть, что часть ошибок модель не смогла заметиь, а часть не может заметить в принципе. Так что полученное значение выглядит правдоподобно.

2). Среднее количество исправлений в предложениях, в которых хоть что-то было исправлено.

In [16]:
num_corrections = []
for key, value in data.items():
    num_corrections.append(len(value))
num_corrections = np.array(num_corrections)
print(f'Среднее количество исправлений: {np.mean(num_corrections):.3f}')

Среднее количество исправлений: 2.125


В среднем имеем примерно два исправления на предложение.

3). Доля случаев, когда использовано максимальное число итераций.

In [17]:
print(f'Доля случаев, когда использовано максимальное число итерераций: '
      f'{np.mean(num_corrections == 5):.3f}')

Доля случаев, когда использовано максимальное число итерераций: 0.105


4). Доля случаев, когда нет корректного токена в списке кандидатов.

In [18]:
exist_correct_candidate = []
for key, value in data.items():
    for item in value:
        answer = item['answer']
        candidates = [x['token'] for x in item['candidates']]
        exist_correct_candidate.append(answer in candidates)
        
print(f'Доля случаев, когда нет корректного токена в списке: '
      f'{1-np.mean(exist_correct_candidate):.3f}')

Доля случаев, когда нет корректного токена в списке: 0.033


5). Среднее количество кандидатов.

In [19]:
num_candidates = []
for key, value in data.items():
    for item in value:
        num_candidates.append(len(item['candidates']))
print(f'Среднее количество кандидатов: {np.mean(num_candidates):.3f}')

Среднее количество кандидатов: 19.016


6). Доля случаев, когда надо оставить изначальный токен при изменении.

In [20]:
remain_original = []
for key, value in data.items():
    for item in value:
        answer = item['answer']
        candidates = [x['token'] for x in item['candidates'] if x['is_original']]
        remain_original.append(answer in candidates)
        
print(f'Доля случаев, когда надо оставить изначальный токен: '
      f'{np.mean(remain_original):.3f}')

Доля случаев, когда надо оставить изначальный токен: 0.424


Как видим, изначальный токен надо оставить в достаточно большом количестве случаев.

## Обучение модели

Теперь обучим модель. Будем решать задачу ранжирования, потому что хотим уметь выбирать одного наилучшего кандидата.

В качестве итоговой метрики возьмем accuracy: частоту правильного угадывания верного исправления.

### Подготовка датафрейма

Начнем с создания датафрейма. Каждому случаю ранжирования надо присвоить отдельную группу.

In [21]:
candidate_keys = list(data[0][0]['candidates'][0].keys())
df_dict = {}
df_dict['group'] = []
df_dict['answer'] = []
group_idx = 0
df_dict.update({key: [] for key in candidate_keys})
for key_data, values_data in data.items():
    for i, item in enumerate(values_data):
        for candidate in item['candidates']:
            df_dict['group'].append(group_idx)
            df_dict['answer'].append(item['answer'])
            for key, value in candidate.items():
                df_dict[key].append(value)
        group_idx += 1

In [22]:
df = pd.DataFrame(df_dict)
df['is_correct'] = (df['token'] == df['answer']).astype(int)
df.head()

,group,answer,token,is_title,is_upper,is_lower,is_first,contains_space,contains_hyphen,from_levenshtein_searcher,...,is_current,from_vocabulary,kenlm_left_right_score,kenlm_right_left_score,kenlm_agg_score,margin_kenlm_agg,bert_score_len,bert_score_sum,bert_score_mean,is_correct
0,0,кто бы,ктобы,False,False,True,False,False,False,False,...,True,False,-7.820433,-8.266124,-8.037104,0.000000,2,-12.701331,-6.350665,0
1,0,кто бы,кто ы,False,False,True,False,True,False,True,...,False,True,-9.029816,-9.861761,-9.427470,-1.390366,2,-17.923715,-8.961858,0
2,0,кто бы,кт бы,False,False,True,False,True,False,True,...,False,True,-9.154133,-9.646735,-9.393981,-1.356877,3,-30.302781,-10.100927,0
3,0,кто бы,кто бы,False,False,True,False,True,False,True,...,False,True,-5.504485,-4.735255,-5.090977,2.946127,2,-7.410320,-3.705160,1
4,0,кто бы,к обы,False,False,True,False,True,False,True,...,False,True,-10.980565,-12.116825,-11.520746,-3.483642,2,-30.354878,-15.177439,0


In [23]:
groups = df['group'].reset_index(drop=True).unique()

### `BertScorer`

Сначала посмотрим, какое качество можно получить, используя уже известный нам `BertScorer`.

In [24]:
succ_predictions = []
for group in groups:
    df_group = df[df.group == group]
    scores = df_group['bert_score_mean']
    prediction_idx = np.argmax(scores)
    succ_predictions.append(
        df_group.answer.iloc[prediction_idx] 
        == df_group.token.iloc[prediction_idx]
    )

succ_predictions = np.array(succ_predictions)
accuracy = np.mean(succ_predictions)

In [25]:
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.508


Видим не очень хорошее значение. Ниже мы увидим, что его можно сильно улучшить.

### Ranking SVM

Попробуем pair-wise подход, а именно ranking SVM. Для каждой позиции мы знаем какой из кандидатов (если, конечно, оптимальный кандидат есть в списке) является наилучшим. В таком случае в паре кандидатов, где один правильным мы знаем какой из них лучше &mdash; так и построим обучающую выборку.

Следует вспомнить, что ranking SVM будет обучаться на разницах признаковых векторов и предсказывать класс такого "разностного" объекта. Для применения этой модели надо будет запустить ее на всех признаковых векторах и найти самое большое значение decision function (это следствие линейности модели).

#### Подготовка данных

Теперь следует создать общий dataframe для попарного подхода. Таргет равен $1$, если кандидат, из которого вычитают строго лучше вычитаемого и $0$ если ситуация обратная (случай, когда не знаем какой из кандидатов лучше не рассматриваем).

Также следует заметить, что признаки: `is_title`, `is_upper`, `is_lower`, `is_first` в нашем подходе оказываются бесполезными, потому что они всегда будут нулевыми (внутри одной группы они у всех одинаковы). Тем не менее, мы их оставим для удобства инференса.

In [26]:
svm_keys = candidate_keys + ['is_correct']
svm_keys.remove('token')

In [27]:
df_svm = pd.DataFrame(columns=svm_keys + ['group'])
for group in df.group.unique():
    cur_data = df[df.group == group]
    cur_correct = cur_data[cur_data.is_correct.astype(bool)][svm_keys]
    cur_incorrect = cur_data[~cur_data.is_correct.astype(bool)][svm_keys]
    if not cur_correct.empty:
        negative_data = (cur_incorrect.astype(float) 
                         - cur_correct.iloc[0].astype(float))
        positive_data = -negative_data
        negative_data['group'] = group
        positive_data['group'] = group
        negative_data['is_correct'] = 0
        positive_data['is_correct'] = 1
        df_svm = df_svm.append(negative_data)
        df_svm = df_svm.append(positive_data)

df_svm = df_svm.astype(float)

Переименуем колонки, чтобы было понятнее, что представляет из себя этот датасет.

In [28]:
df_svm['is_better'] = df_svm['is_correct'].astype(int)
df_svm.drop(columns=['is_correct'], inplace=True)

In [29]:
df_svm.head()

,is_title,is_upper,is_lower,is_first,contains_space,contains_hyphen,from_levenshtein_searcher,from_phonetic_searcher,from_handcode_searcher,is_original,...,from_vocabulary,kenlm_left_right_score,kenlm_right_left_score,kenlm_agg_score,margin_kenlm_agg,bert_score_len,bert_score_sum,bert_score_mean,group,is_better
0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,1.0,...,-1.0,-2.315948,-3.530869,-2.946127,-2.946127,0.0,-5.291010,-2.645505,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-3.525331,-5.126506,-4.336493,-4.336493,0.0,-10.513395,-5.256697,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-3.649648,-4.911480,-4.303004,-4.303004,1.0,-22.892460,-6.395767,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-5.476080,-7.381570,-6.429769,-6.429769,0.0,-22.944558,-11.472279,0.0,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-8.111958,-10.026857,-9.075175,-9.075175,1.0,-37.236293,-11.177044,0.0,0


Посмотрим на размер датасета:

In [30]:
df_svm.shape

(100128, 21)

Достаточно большой.

Преобразуем данные к виду, с которым будем работать в sklearn.

In [31]:
to_drop = [
    'is_better', 'group'
]
X = df_svm.drop(columns=to_drop).reset_index(drop=True)
groups_svm = df_svm['group'].reset_index(drop=True)
y = df_svm['is_better'].reset_index(drop=True)

In [32]:
X.head()

,is_title,is_upper,is_lower,is_first,contains_space,contains_hyphen,from_levenshtein_searcher,from_phonetic_searcher,from_handcode_searcher,is_original,is_current,from_vocabulary,kenlm_left_right_score,kenlm_right_left_score,kenlm_agg_score,margin_kenlm_agg,bert_score_len,bert_score_sum,bert_score_mean
0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,1.0,1.0,-1.0,-2.315948,-3.530869,-2.946127,-2.946127,0.0,-5.291010,-2.645505
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.525331,-5.126506,-4.336493,-4.336493,0.0,-10.513395,-5.256697
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.649648,-4.911480,-4.303004,-4.303004,1.0,-22.892460,-6.395767
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.476080,-7.381570,-6.429769,-6.429769,0.0,-22.944558,-11.472279
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.111958,-10.026857,-9.075175,-9.075175,1.0,-37.236293,-11.177044


#### Обучение модели и ее тестирование

Теперь обучим модель и протестируем ее. Для этого будем делать кросс-валидацию по номерам группы, а на оставшихся для тестирования группах будем проверять, что самое большое предсказание получает лучший кандидат.

In [33]:
kf = KFold()
accuracy_values = []
for groups_indices_train, groups_indices_test in kf.split(groups):
    groups_train = groups[groups_indices_train]
    X_train = X.loc[groups_svm.isin(groups_train)]
    y_train = y.loc[groups_svm.isin(groups_train)]
    model = Pipeline(
        [('scaler', StandardScaler()), ('svc', LinearSVC(random_state=42))]
    )
    model = LinearSVC(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    
    # test accuracy of prediction
    succ_predictions = []
    for group in groups[groups_indices_test]:
        df_group = df[df.group == group]
        X_test = df_group[X_train.columns]
        scores = model.decision_function(X_test)
        prediction_idx = np.argmax(scores)
        succ_predictions.append(
            df_group.answer.iloc[prediction_idx] 
            == df_group.token.iloc[prediction_idx]
        )
        
    succ_predictions = np.array(succ_predictions)
    accuracy_values.append(np.mean(succ_predictions))

/home/mrgeekman/Programs/anaconda3/envs/bs_research/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mrgeekman/Programs/anaconda3/envs/bs_research/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mrgeekman/Programs/anaconda3/envs/bs_research/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mrgeekman/Programs/anaconda3/envs/bs_research/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/mrgeekman/Programs/anacond

Посмотрим на среднее значение accuracy:

In [34]:
accuracy = np.mean(accuracy_values)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.897


Если убрать признаки `bert_score_len`, `bert_score_sum`, `bert_score_mean`, то получилось значение $0.887$. Видим, что результат хуже примерно на $1\%$.

Значение получилось гораздо лучше, чем при использовании только `bert_score`.

Обучим модель на всех данных и сохраним ее.

In [35]:
model = Pipeline(
        [('scaler', StandardScaler()), ('svc', LinearSVC(random_state=42))]
)
model.fit(X, y)

/home/mrgeekman/Programs/anaconda3/envs/bs_research/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('scaler', StandardScaler()),
                ('svc', LinearSVC(random_state=42))])

In [ ]:
with open(os.path.join(MODEL_PATH, 'candidate_scorer', 'svm.bin'), 'wb') as ouf:
    pickle.dump(model, ouf)

Посмотрим на веса коэффициентов в модели:

In [36]:
for coef, feature_name in zip(model['svc'].coef_.ravel(), X.columns.tolist()):
    print(f'{feature_name}: {coef:.3f}')

is_title: 0.000
is_upper: 0.000
is_lower: 0.000
is_first: 0.000
contains_space: 0.409
contains_hyphen: 0.164
from_levenshtein_searcher: 1.046
from_phonetic_searcher: 0.674
from_handcode_searcher: 0.695
is_original: 2.424
is_current: -0.136
from_vocabulary: 0.054
kenlm_left_right_score: 1.007
kenlm_right_left_score: 1.226
kenlm_agg_score: -0.072
margin_kenlm_agg: -0.072
bert_score_len: 0.022
bert_score_sum: 0.022
bert_score_mean: 0.834


1. Как ожидалось, модель проигнорировала первые четыре признака.
2. Модель дает высокую оценку коэффициенту перед индикатором изначального токена. То есть модель дает большое предпочтение изначальному токену.
3. Модель выше оценивает, что исправление пришло из levenshtein searcher, хотя казалось, что самый большой вклад должен иметь кандидат из handcode searcher, ведь он почти всегда дает правильное исправление.
4. Модель сильнее реагирует на наличие пробелов, чем на наличие дефисов.
5. Среди всех скоров от BERT наибольшее влияние оказывает скор после аггрегации средним. Тем не менее, даже так этот коэффициент по модулю меньше коэффициентов, соответствующих kenlm.
6. Модель практически проигнорировала признаки из kenlm помимо скоров от левой и правой моделей.

### CatBoost

Теперь попробуем обучить модель CatBoost. Такая модель уже сможет утилизировать первые те признаки, которые проигнорировала предыдущая. Логичным кажется оптимизировать метрику MRR, так как внутри каждой группы имеем всего один релевантный объект.

#### Подготовка данных

Надо создать `Pool` для обучения.

In [37]:
to_drop = [
    'group', 'is_correct', 'answer', 'token'
]
pool_catboost = Pool(
    data=df.drop(columns=to_drop),
    label=df['is_correct'],
    group_id=df['group']
)

#### Обучение модели и ее тестирование

Теперь обучим модель и протестируем ее при помощи кросс-валидации.

In [38]:
def test_catboost_model(params):
    kf = KFold()
    accuracy_values = []
    tree_counts = []
    for groups_indices_train, groups_indices_test in kf.split(groups):
        groups_train = groups[groups_indices_train]
        df_train = df[df['group'].isin(groups_train)]
        pool_train = Pool(
            data=df_train.drop(
                columns=to_drop
            ),
            label=df_train['is_correct'],
            group_id=df_train['group']
        )

        groups_test = groups[groups_indices_test]
        df_test = df[df['group'].isin(groups_test)]
        pool_test = Pool(
            data=df_test.drop(
                columns=to_drop
            ),
            label=df_test['is_correct'],
            group_id=df_test['group']
        )

        model = CatBoost(params)
        model.fit(pool_train, eval_set=pool_test)
        tree_counts.append(model.tree_count_)
        accuracy_values.append(
            model.evals_result_[
                'validation'
            ]['PrecisionAt:top=1'][model.tree_count_-1]
        )
    return np.mean(accuracy_values), np.mean(tree_counts)

In [ ]:
params = {
    'loss_function': 'PairLogit',
    'eval_metric': 'PrecisionAt:top=1',
    'random_seed': 42,
    'verbose': False
}

test_catboost_model(params)

Посмотрим на среднее значение accuracy для разных функций потерь:

* RMSE: $0.897$
* QueryRMSE: $0.902$
* PairLogit: $0.916$ 
* YetiRank: $0.914$
* YetiRankPairwise: $0.897$

Получившиеся значения лучше значений, полученных в SVM. Как видим, лучше всего взять в качестве лосса PairLogit.

То же самое, но если убрать признаки `bert_score_len`, `bert_score_sum`, `bert_score_mean`:

* RMSE: $0.891$
* QueryRMSE: $0.898$
* PairLogit: $0.906$ 
* YetiRank: $0.901$
* YetiRankPairwise: $0.883$

Как видим, без признаков BERT опять падение примерно на $1\%$.

Обучим модель на всех данных и сохраним ее.

In [39]:
params = {
    'loss_function': 'PairLogit',
    'random_seed': 42,
    'verbose': False
}

model = CatBoost(params)
model.fit(pool_catboost)
model.shrink(350)

In [ ]:
save_path = os.path.join(MODEL_PATH, 'candidate_scorer', 'catboost_no_bert.cbm')
model.save_model(save_path)

Посмотрим на значимости различных признаков в модели:

In [40]:
feature_importances = model.get_feature_importance(pool_catboost)
feature_names = model.feature_names_
for feature_name, importance in zip(feature_names, feature_importances):
    print(f'{feature_name}: {importance*1000:.3f}')

is_title: 0.036
is_upper: 0.025
is_lower: 0.077
is_first: 0.324
contains_space: 0.478
contains_hyphen: 0.232
from_levenshtein_searcher: 1.834
from_phonetic_searcher: 2.475
from_handcode_searcher: 0.062
is_original: 65.921
is_current: 0.050
from_vocabulary: 0.152
kenlm_left_right_score: 1.053
kenlm_right_left_score: 0.652
kenlm_agg_score: 1.604
margin_kenlm_agg: 11.174
bert_score_len: 0.454
bert_score_sum: 0.558
bert_score_mean: 4.472


1. Модель практически проигнорировала первые 3 признака.
2. Модель обращает внимание на то, является ли токен первым в предложении.
3. Модель дает высокую оценку коэффициенту перед индикатором изначального токена. То есть модель дает большое предпочтение изначальному токену. В то же время модель почти не обращает внимание на то текущий это токен или нет.
4. Модель выше оценивает, что исправление пришло из phonetic searcher, затем levenshtein searcher и самое наименьшее влияние имеет handcode searcher.
5. Модель оценивает влияние дефисов и пробелов примерно в одинаковом порядке.
6. Среди скоров kenlm наибольшее внимание уделено скору, который отвечает за превосходство рассматриваемого кандидата над используемым на данный момент.
7. Среди скоров BERT наибольшее внимание уделено аггрегации при помощи среднего.

## Выводы

1. В этом ноутбуке была собрана обучающая выборка для candidate scorer, который использует другие признаки помимо скоров из BERT.
2. Было показано, что модель, основанная только на `bert_score` сильно уступает моделям, которые утилизируют также другие признаки.
3. Была обучена модель ranking SVM с точностью около $90.0\%$.
4. Была обучена модель CatBoost с точностью около $91.5\%$.